In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install deap

In [ ]:
import random
import operator
import csv
import itertools
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
import pandas as pd 
df = pd.read_csv("../input/advertising-cmaes/advertising.csv",header=None)
df.drop([4,5,7,8], axis = 1, inplace=True)
df.columns = [0,1,2,3,4,5]
print(df.head(5))


In [ ]:
df = df.drop(0)
print(df.head(5))

In [ ]:
 
print(df.head(5))

In [ ]:
print(df.columns)
df[5] = df[5].astype('int').astype('bool')
print(df.head(10))

In [ ]:
train, test = train_test_split(df, test_size=0.2)
print(train.shape)
print(train.head(10))
train_data = train.values.tolist()
print(test.head(10))

In [ ]:
train_data_new = list(list(float(elem) if elem != '?' else float(0) for elem in row) for row in train_data)
print(train_data_new)

In [ ]:
print(train)

In [ ]:
train_res = train[5]
test_res = test[5]
print(train_res)

In [ ]:
print(test_res)

In [ ]:
pset = gp.PrimitiveSetTyped("MAIN", itertools.repeat(float, 4), bool, "IN")

pset.addPrimitive(operator.and_, [bool, bool], bool)
pset.addPrimitive(operator.or_, [bool, bool], bool)
pset.addPrimitive(operator.not_, [bool], bool)

In [ ]:
def protectedDiv(left, right):
    try: return left / right
    except ZeroDivisionError: return 1

In [ ]:
pset.addPrimitive(operator.add, [float,float], float)
pset.addPrimitive(operator.sub, [float,float], float)
pset.addPrimitive(operator.mul, [float,float], float)
pset.addPrimitive(protectedDiv, [float,float], float)

In [ ]:

def if_then_else(input, output1, output2):
    if input: return output1
    else: return output2

In [ ]:
pset.addPrimitive(operator.lt, [float, float], bool)
pset.addPrimitive(operator.eq, [float, float], bool)
pset.addPrimitive(if_then_else, [bool, float, float], float)
pset.addTerminal(False, bool)
pset.addTerminal(True, bool)

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

In [ ]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [ ]:
def evalTrainbase(individual):
    func = toolbox.compile(expr=individual)
    train_data_2_sample = random.sample(train_data_new, 200)
    result = sum(bool(func(*val[:4])) is bool(val[4]) for val in train_data_2_sample)
    return result,

In [ ]:
toolbox.register("evaluate", evalTrainbase)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [ ]:
random.seed(10)
pop = toolbox.population(n=100)

In [ ]:

hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)
    
algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 50, stats, halloffame=hof)

print(pop, stats, hof)

In [ ]:
hof

In [ ]:
pop[0]

In [ ]:
print(test.head(5))

In [ ]:
test_data = test.values.tolist()
print(test_data)

In [ ]:
test_data_new = list(list(float(elem) if elem != '?' else float(0) for elem in row) for row in test_data)
print(test_data_new)

In [ ]:
def evalTest(individual):
    func = toolbox.compile(expr=individual)
    result = sum(1 if (func(*val[:4]) == bool(val[4])) else 0 for val in test_data_new)
    return result

In [ ]:
for ind in pop:
    print(evalTest(ind))

In [ ]:
toolbox.compile(expr=pop[0])

In [ ]:
print(test_data_new)

In [ ]:
def predict(individual):
    func = toolbox.compile(expr=individual)
    result = list(func(*val[:4]) for val in test_data_new)
    return result

In [ ]:
accuracy_score(test_res, predict(pop[0]))

In [ ]:
confusion_matrix_val = confusion_matrix(test_res, predict(pop[0]))
labels = ['Benign', 'Malignant']

# Create the plot figure 
fig = plt.figure()

# Comfusion Matrix subplot
subplt = fig.add_subplot(111)
csubplt = subplt.matshow(confusion_matrix_val)

plt.title('Confusion matrix :')
fig.colorbar(csubplt)

subplt.set_xticklabels([''] + labels)
subplt.set_yticklabels([''] + labels)

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
nodes, edges, labels = gp.graph(pop[0])

In [ ]:

!apt-get install graphviz libgraphviz-dev pkg-config

In [ ]:
!pip install pygraphviz

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

g = nx.Graph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
pos = graphviz_layout(g, prog="dot")

nx.draw_networkx_nodes(g, pos)
nx.draw_networkx_edges(g, pos)
nx.draw_networkx_labels(g, pos, labels)
plt.show()